# AstroPy: Tables

The astropy [Table](http://docs.astropy.org/en/stable/table/index.html) class provides an extension of NumPy structured arrays for storing and manipulating heterogeneous tables of data. A few notable features of this package are:

For more information about the features presented below, you can read the
[astropy.table](http://docs.astropy.org/en/stable/table/index.html) docs.



In [1]:
from astropy.table import Table
import numpy as np

Creating tables
---------------

There is great deal of flexibility in the way that a table can be initially constructed:

- Read an existing table from a file or web URL
- Add columns of data one by one
- Add rows of data one by one
- From an existing data structure in memory (e.g. a list of dictionaries)
  
See the documentation section on [Constructing a table](http://astropy.readthedocs.org/en/stable/table/construct_table.html) for the gory details and plenty of examples.

In [2]:
t = Table()
t['name'] = ['source 1', 'source 2', 'source 3', 'source 4']
t['flux'] = [1.2, 2.2, 3.1, 4.3]

### Looking at your table

t.pprint() shows a nice representation of the table:

In [3]:
t.pprint()

  name   flux
-------- ----
source 1  1.2
source 2  2.2
source 3  3.1
source 4  4.3


To get the table column names and data types using the `colnames` and `dtype` properties:

In [4]:
t.colnames

['name', 'flux']

In [5]:
t.dtype

dtype([('name', 'S8'), ('flux', '<f8')])

### Accessing parts of the table

We can access the columns and rows by name and number.  Notice that the outputs are `Column`, `Row` or `Table` objects depending on the context.

In [6]:
t['flux']  # Flux column (notice meta attributes)

1.2
2.2
3.1
4.3


In [7]:
t['flux'][1]  # Row 1 of flux column

2.2000000000000002

In [8]:
t[1]  # Row obj for with row 1 values

name,flux
str8,float64
source 2,2.2


In [9]:
t[1]['flux']  # Flux column of row 1

2.2000000000000002

In [10]:
t[1:3]  # 2nd and 3rd rows in a new table

name,flux
str8,float64
source 2,2.2
source 3,3.1


In [11]:
t[[1, 3]]

name,flux
str8,float64
source 2,2.2
source 4,4.3


**One of the most powerful concepts is using boolean selection masks to filter tables**

In [12]:
mask = t['flux'] > 3.0  # Define boolean mask for all flux values > 3
t[mask]  # Create a new table with only the "high flux" sources

name,flux
str8,float64
source 3,3.1
source 4,4.3


### Modifying the table

Once the table exists with defined columns there are a number of ways to modify the table in place.  These are fully documented in the section [Modifying a Table](http://astropy.readthedocs.org/en/stable/table/modify_table.html#modifying-a-table).

To give a couple of simple examples, you can add rows with the [add_row()](http://astropy.readthedocs.org/en/stable/api/astropy.table.table.Table.html#astropy.table.table.Table.add_row) method or add new columns using dict-style assignment:

In [13]:
t.add_row(('source 5', 10.1))  # Add a new source at the end
t['logflux'] = np.log10(t['flux'])  # Compute the log10 of the flux
t

name,flux,logflux
str8,float64,float64
source 1,1.2,0.0791812460476
source 2,2.2,0.342422680822
source 3,3.1,0.491361693834
source 4,4.3,0.63346845558
source 5,10.1,1.00432137378


Notice that the `logflux` column really has too many output digits given the precision of the input values.  We can fix this by setting the format using normal Python formatting syntax:

In [14]:
t['flux'].format = '%.2f'
t['logflux'].format = '%.2f'
t

name,flux,logflux
str8,float64,float64
source 1,1.20,0.08
source 2,2.20,0.34
source 3,3.10,0.49
source 4,4.30,0.63
source 5,10.10,1.00


### Converting the table to numpy

Sometimes you may not want or be able to use a `Table` object and prefer to work with a plain numpy array.  This is easily done by passing the table to the `np.array()` constructor.  This makes a copy of the data.

In [15]:
np.array(t)

array([('source 1',   1.2,  0.07918125), ('source 2',   2.2,  0.34242268),
       ('source 3',   3.1,  0.49136169), ('source 4',   4.3,  0.63346846),
       ('source 5',  10.1,  1.00432137)], 
      dtype=[('name', 'S8'), ('flux', '<f8'), ('logflux', '<f8')])

Masked tables
-------------

In [ ]:
t2 = Table([['x', 'y', 'z'], 
            [1.1, 2.2, 3.3]],
           names=['name', 'value'],
           masked=True)
print t2

In [ ]:
t2['value'].mask = [False, True, False]

In [ ]:
print t2

In [ ]:
t2['value'].fill_value = -99
print t2.filled()

High-level table operations
----------------------------

So far we've just worked with one table at a time and viewed that table as a monolithic entity.  Astropy also supports high-level [Table operations](http://astropy.readthedocs.org/en/stable/table/operations.html) that manipulate multiple tables or view one table as a collection of sub-tables (groups).

 Documentation	                                                                         | Description
---------------------------------------------------------------------------------------- |-----------------------------------------
[Grouped operations](http://astropy.readthedocs.org/en/stable/table/operations.html#id2) | Group tables and columns by keys
[Stack vertically](http://astropy.readthedocs.org/en/stable/table/operations.html#id3)   | Concatenate input tables along rows
[Stack horizontally](http://astropy.readthedocs.org/en/stable/table/operations.html#id4) | Concatenate input tables along columns
[Join](http://astropy.readthedocs.org/en/stable/table/operations.html#join)              | Database-style join of two tables

Here we'll just introduce the join operation but go into more detail on the others in the exercises.

In [ ]:
from astropy.table import join

Now recall our original table `t`:

In [ ]:
print t

Now say that we now got some additional flux values from a different reference for a different, but overlapping sample of sources:

In [ ]:
t2 = Table()
t2['name'] = ['source 1', 'source 3', 'source 8']
t2['flux2'] = [1.4, 3.5, 8.6]
print t2

Now we can get a master table of flux measurements which are joined matching the values the `name` column.  This includes every row from each of the two tables, which is known as an **`outer`** join.

In [ ]:
t3 = join(t, t2, keys=['name'], join_type='outer')
print t3

In [ ]:
np.mean(t3['flux2'])

Alternately we could choose to keep only rows where both tables had a valid measurement using an **`inner`** join:

In [ ]:
join(t, t2, keys=['name'], join_type='inner')

Writing data
------------

In [ ]:
t3.write('test.fits', overwrite=True)

In [ ]:
t3.write('test.txt', format='ascii')

Reading data
------------

You can read data using the [Table.read()](http://astropy.readthedocs.org/en/stable/api/astropy.table.table.Table.html#astropy.table.table.Table.read) method:

In [ ]:
t4 = Table.read('test.fits')
t4